In [1]:
import json
import requests as rq
import pandas as pd
import time

apikey = '9c594b29-ad69-4833-b88b-db5cdfa30683'
apikey2= '12504de4-1fe7-4700-90ff-13336ef9cc2a'

import pymongo
from pymongo import MongoClient
client = MongoClient()
db = client.lol_match_db
collection = db.match_list_2016_5x5

In [10]:
list_summoners = pd.read_excel('LVPSummoner.xlsx',
                      sheetname = 'table_output',
                      header = 0,
                      index_col = 0,
                      parse_cols = "A",
                      convert_float = True)
len(list_summoners)

440

In [11]:
print list_summoners.index[0]

19459077


In [25]:
cursor = db.match_list_2016_5x5.find({"_id": str(list_summoners.index[0])})
for document in cursor:
    print document['matches']['totalGames']

69


In [18]:
i=0
summoner_list = []
number_matches = []
match_id_list = []

while (i<len(list_summoners)):
    cursor = db.match_list_2016_5x5.find({"_id": str(list_summoners.index[i])})
    for document in cursor:
        summoner_list.append(document['_id'])
        number_matches.append(document['matches']['totalGames'])
        j=0
        while (j<document['matches']['totalGames']):
            match_id_list.append(str(document['matches']['matches'][j]['matchId']))
            j=j+1
    i=i+1
        
print len(summoner_list)
print len(number_matches)
print len(match_id_list)


440
440
228117


In [20]:
d = {'SummonerID' : summoner_list,'totalGames' : number_matches}
table_summoner_totalgames = pd.DataFrame(d)

In [26]:
table_summoner_totalgames.head()

,SummonerID,totalGames
0,19459077,69
1,20501992,7
2,23407188,206
3,27281117,37
4,28306135,111


In [28]:
table_match_id_list = pd.DataFrame(match_id_list)
table_match_id_list.head()

,0
0,2853586095
1,2807676541
2,2807350386
3,2804711583
4,2795203055


In [27]:
#IT WORKS!
#Now, let's move this to an excel file

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('SummonerTotalGames.xlsx', engine='xlsxwriter')
# Convert the dataframe to an XlsxWriter Excel object.
table_summoner_totalgames.to_excel(writer, sheet_name='table_summoner_totalgames')
# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [29]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('MatchIDList.xlsx', engine='xlsxwriter')
# Convert the dataframe to an XlsxWriter Excel object.
table_match_id_list.to_excel(writer, sheet_name='match_id_list')
# Close the Pandas Excel writer and output the Excel file.
writer.save()

# A PARTIR D'AQUÍ ÉS PER RETRIEVE MATCH DATA

{u'matches': [{u'lane': u'BOTTOM', u'champion': 81, u'platformId': u'EUW1', u'season': u'SEASON2016', u'region': u'EUW', u'matchId': 2556336889L, u'queue': u'TEAM_BUILDER_DRAFT_RANKED_5x5', u'role': u'DUO_CARRY', u'timestamp': 1457636026933L}, {u'lane': u'BOTTOM', u'champion': 12, u'platformId': u'EUW1', u'season': u'SEASON2016', u'region': u'EUW', u'matchId': 2556239142L, u'queue': u'TEAM_BUILDER_DRAFT_RANKED_5x5', u'role': u'DUO_SUPPORT', u'timestamp': 1457631940432L}, {u'lane': u'BOTTOM', u'champion': 40, u'platformId': u'EUW1', u'season': u'SEASON2016', u'region': u'EUW', u'matchId': 2543767303L, u'queue': u'TEAM_BUILDER_DRAFT_RANKED_5x5', u'role': u'DUO_SUPPORT', u'timestamp': 1456938853896L}, {u'lane': u'BOTTOM', u'champion': 25, u'platformId': u'EUW1', u'season': u'SEASON2016', u'region': u'EUW', u'matchId': 2526397764L, u'queue': u'TEAM_BUILDER_DRAFT_RANKED_5x5', u'role': u'DUO_SUPPORT', u'timestamp': 1455841269661L}, {u'lane': u'MID', u'champion': 134, u'platformId': u'EUW1', 

In [10]:
len(document['matches']['matches'])

7

In [11]:
document['matches']['matches'][1]['matchId']

2556239142L

In [15]:
str(document['matches']['matches'][1]['matchId'])

'2556239142'

In [22]:
import sys
filename  = open("log_match_details.txt",'w')
sys.stdout = filename
print "RESPONSES"

In [29]:
r = rq.get('https://euw.api.pvp.net/api/lol/euw/v2.2/match/' +str(document['matches']['matches'][1]['matchId'])+'?includeTimeline=TRUE&api_key=' + apikey)
r.text

u'{\n\t"status": {\n\t\t"message": "429",\n\t\t"status_code": 429\n\t}\n}'

In [32]:
#retrieve matches
i=0
while i < len(document['matches']['matches']):  
    #just for updates
    print i
    result = db.match_details_test.remove({ "_id": str(document['matches']['matches'][i]['matchId'])})
    
    #get all matches for a summoner id
    #WARNING: let's do only one request every two seconds
    r = rq.get('https://euw.api.pvp.net/api/lol/euw/v2.2/match/' +str(document['matches']['matches'][i]['matchId'])+'?includeTimeline=TRUE&api_key=' + apikey)
    #check response code
    while (r.status_code <> 200):
        time.sleep(4)
        r = rq.get('https://euw.api.pvp.net/api/lol/euw/v2.2/match/' +str(document['matches']['matches'][i]['matchId'])+'?includeTimeline=TRUE&api_key=' + apikey)
        
    print str(document['matches']['matches'][1]['matchId']) +' '+ str(r.status_code)
    result = db.match_details_test.insert_one({"_id": str(document['matches']['matches'][i]['matchId']), "details":json.loads(r.text)})
    print 'Inserted at:' + result.inserted_id
    #insert into match_list_2016(contains all matches for Season 2016 for any particular summoner) using
    #the summoner ID as DB Key    
    #increase and wait    
    i = i + 1
    time.sleep(2)